In [1]:
import numpy as np
import pandas as pd
import torch
import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor
from sklearn.preprocessing import MinMaxScaler
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations


#from math import ceil
#from google.colab import drive
#drive.mount('/content/gdrive')
import torch.nn.init as init
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import torch.nn.functional as Fun
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features = list()
with open("/home/c/cl237/TenBCD/UCI HAR/data/features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('# of Features: {}'.format(len(features)))

# of Features: 561


## get train data

In [4]:
pd_X_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_train.txt', delim_whitespace=True, header=None)
pd_X_train.columns = features
pd_y_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_train.txt', delim_whitespace=True, names=['Activity'])

N = len(pd_X_train)

K = 6

x = pd_X_train.values
y = pd_y_train.values

X_train=torch.tensor(x,dtype = torch.float32,device=device)
y_train=torch.tensor(y,dtype = int) #dtype=torch.int,device=device)

y_train = torch.flatten(y_train)

X_train = torch.t(X_train)

y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train-1, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

## get test data

In [5]:
pd_X_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_test.txt', delim_whitespace=True, header=None)
pd_X_test.columns = features
pd_y_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_test.txt', delim_whitespace=True, names=['Activity'])

N_test = len(pd_X_test)

x = pd_X_test.values
y = pd_y_test.values

X_test=torch.tensor(x,dtype = torch.float32, device=device)
y_test=torch.tensor(y,dtype = int)

y_test = torch.flatten(y_test)

X_test = torch.t(X_test)

y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test-1, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [6]:
def updateV(U1,U2,W,b,rho,gamma): 
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho): 
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [8]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/rank40_100times.csv')
niter = 500
rank = 80
tau = 5
alpha = 1
gamma = 0.5
rho = 0.5

#alpha1 = alpha2 = alpha3 = alpha4 = alpha5 = alpha6 = alpha7 = alpha8 = alpha9 = alpha10 = alpha
#for rank in (35,): #(25,30,35)(100,180,220,260,300,340,380)(20,60,100,140,180,220,260,300,340,380)
#    for tau in (400,500): #(300,400,500)(10,50,100,200,300)(10,50,100,200,300)
#        for gamma in (0.5,0.8,2): #(0.5,0.8,2)(0.5,0.8)(0.5,1,1.5,2,3)
            #gamma1 = gamma2 = gamma3 = gamma4 = gamma5 = gamma
#            for rho in (0.5,0.8,2): #(0.5,0.8)(1,2)
                #rho1 = rho2 = rho3 = rho4 = rho5= rho
#                for alpha in (0.5,1,1.5,2):
#                    print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)
                    #print('Compression Ratio', ((1024*28*28+10*1024+(8*(rank)+32*np.square(rank))*2)/(1024*28*28+10*1024+1024*1024*2)), (8*(rank)+32*np.square(rank))*2/(1024*1024*2))
        
print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)



for Out_iter in range(1):
    rank_initial = 400
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)

    d0 = 561 #561 =3*11*17
    d1 =  1024    # 2^10
    d2 =  1024  
    d3 = 1024
    d4 = 512      # 2^9
    d5 = 6 

    W1 = 0.2*init.xavier_normal_(torch.empty(d1, d0, device=device), gain=1.0)
    #W1 = 0.01*torch.randn(d1, d0, device=device)
    W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
    W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    W2 = 0.2*init.xavier_normal_(torch.empty(d2, d1, device=device), gain=1.0)
    W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray      
    b2 = 0*torch.ones(d2, 1, device=device)


    W3 = 0.2*init.xavier_normal_(torch.empty(d3, d2, device=device), gain=1.0)
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    W4 = 0.2*init.xavier_normal_(torch.empty(d4, d3, device=device), gain=1.0)
    W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2)) # 8 number of 4s, 2 number of 8s
    W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    factors4 = tensor_train(W4_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W4_tl_tensor_rec = tt_to_tensor(factors4)
    b4 = 0*torch.ones(d4, 1, device=device)


    W5 = 0.2*init.xavier_normal_(torch.empty(d5, d4, device=device), gain=1.0)
    b5 = 0*torch.ones(d5, 1, device=device)

    # W4 = 0.01*torch.randn(d4, d3, device=device)
    # W4_torch_tensor = W4.reshape((50,30,K))
    # W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    # core4, tucker_factors4 = tucker(W4_tl_tensor, rank=(40,28,K), init='svd', tol=10e-5, random_state=12345)
    # W4_tl_tensor_rec = tl.tucker_to_tensor((core4, tucker_factors4))
    # b4 = 0.1*torch.ones(d4, 1, device=device)

    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = nn.ReLU()(U4)
    U5 = torch.addmm(b5.repeat(1, N), W5, V4)
    V5 = U5 
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4



    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

 # update for last layer
        # update V4
        V5 = (y_one_hot + gamma*U5 + alpha*V5)/(1 + gamma + alpha)

        # update U4 
        U5 = (gamma*V5 + rho*(torch.mm(W5,V4) + b5.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W5, b5 = updateWb_org(U5,V4,W5,b5,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)



  # update for 4th layer
        # update V3
        V4 = updateV(U4,U5,W5,b5,rho,gamma)

        # update U3
        U4 = relu_prox(V4,(rho*torch.addmm(b4.repeat(1,N), W4, V3) + alpha*U4)/(rho + alpha),(rho + alpha)/gamma,d4,N)

        # update W3 and b3
        W4, b4 = updateWb(U4,V3,W4,b4,W4_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2))
        W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors4 = tensor_train(W4_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W4_tl_tensor_rec = tt_to_tensor(factors4)

  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)

 # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy()) 
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


# update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWb(U1,X_train,W1,b1,W1_tl_tensor_rec, alpha,rho,tau)

        # G update
        W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
        W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W1_tl_tensor_rec = tt_to_tensor(factors1)




 # prediction for trainning data
        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_train))
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train))
        a4_train = nn.ReLU()(torch.addmm(b4.repeat(1, N), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b5.repeat(1, N), W5, a4_train), dim=0)
        # check argmax and addmm, dim=0

 #Prediction for test data
        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_test))
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test))
        a4_test = nn.ReLU()(torch.addmm(b4.repeat(1, N_test), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_test))
        pred_test = torch.argmax(torch.addmm(b5.repeat(1, N_test), W5, a4_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        # # a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        # # a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), W2, a1_train))

        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), W2, a1_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)




   #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V5,y_one_hot,2),2).cpu().numpy()
        # torch.pow:Takes the power of each element in input with exponent and returns a tensor with the result.
        # torch.disk: Returns the p-norm of (input - other)
        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b5.repeat(1,N), W5, V4),U5,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,nn.ReLU()(U4),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V5,U5,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W1.reshape((6,22,34,2,2,2,2,2,2,2)),torch.as_tensor(W1_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W4.reshape((4,4,4,4,4,4,4,4,4,2)),torch.as_tensor(W4_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() 
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy() 

        # compute training accuracy
        correct_train = pred == y_train-1
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test-1
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n', 
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors1_shape=[f.shape for f in factors1]
    Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    factors4_shape=[f.shape for f in factors4]
    Sum_of_variables_factors4=sum(list(x*y*z for x,y,z in factors4_shape))

    total_variabels=Sum_of_variables_factors1+Sum_of_variables_factors2+Sum_of_variables_factors3+Sum_of_variables_factors4

    CR_1=((total_variabels)+(d4*d5))/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5)
    CR_2=(total_variabels)/(d0*d1+d1*d2+d2*d3+d3*d4)
    print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(CR_1,CR_2)

#this postion to add new row into existing table
    #df=pd.read_csv('C:/Users/Mark/Desktop/rank40_100times.csv')
    #new_row = {'rank':rank, 'CR_1':CR_1, 'CR_2':CR_2, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
    #           'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
    #           'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1], 'seed':seed} 
    #df=df.append(new_row,ignore_index=True)
    #df.to_csv('C:/Users/Mark/Desktop/rank40_100times.csv',index=False)


filename= "XavierNormal_" + "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/UCI HAR/4 hidden layers/Different Initialization/
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})
#this position to save table into matlab
#df.to_csv('C:/Users/Mark/Desktop/result_compress_123_layer_highrank.csv')
##Changing Folder
 #%cd '/content/gdrive/MyDrive/Colab/Tensor-BCD-for-DNN/Experiments/DataSaved'
 #savemat("ThreeLayer_rank160.mat", {'results': torch.Tensor.numpy(results)})    

rank= 80 tau= 5 gamma= 0.5 rho= 0.5 alpha 1
Train on 7352 samples, validate on 2947 samples
Repeatition 1 Epoch 1 / 500 
 - time: 1.7425446510314941 - sq_loss: 661.6697387695312 - tot_loss: 1023.5006638470877 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 2 / 500 
 - time: 1.3214137554168701 - sq_loss: 294.075439453125 - tot_loss: 475.39059319347143 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 3 / 500 
 - time: 1.315943717956543 - sq_loss: 160.69215393066406 - tot_loss: 261.01051138341427 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 4 / 500 
 - time: 1.3142530918121338 - sq_loss: 86.99430847167969 - tot_loss: 147.26321842707694 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 5 / 500 
 - time: 1.3319859504699707 - sq_loss: 46.784786224365234 - tot_loss: 85.8411285309121 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 6 / 500 
 - time: 1.

Repeatition 1 Epoch 48 / 500 
 - time: 1.2911641597747803 - sq_loss: 0.000612259260378778 - tot_loss: 0.9720702514241566 - acc: 0.8857453754080522 - val_acc: 0.8686800135731252
Repeatition 1 Epoch 49 / 500 
 - time: 1.309067726135254 - sq_loss: 0.000572146091144532 - tot_loss: 0.9318796098377788 - acc: 0.8875136017410229 - val_acc: 0.8730912792670512
Repeatition 1 Epoch 50 / 500 
 - time: 1.302140235900879 - sq_loss: 0.0005350055289454758 - tot_loss: 0.8857553449379338 - acc: 0.8887377584330794 - val_acc: 0.8724126230064473
Repeatition 1 Epoch 51 / 500 
 - time: 1.3069775104522705 - sq_loss: 0.0005007779691368341 - tot_loss: 0.8501487037501647 - acc: 0.891050054406964 - val_acc: 0.8754665761791652
Repeatition 1 Epoch 52 / 500 
 - time: 1.3039112091064453 - sq_loss: 0.0004690583737101406 - tot_loss: 0.8113640344690793 - acc: 0.8932263329706203 - val_acc: 0.8788598574821853
Repeatition 1 Epoch 53 / 500 
 - time: 1.2928962707519531 - sq_loss: 0.00043954490683972836 - tot_loss: 0.778876299

Repeatition 1 Epoch 95 / 500 
 - time: 3.086397171020508 - sq_loss: 4.6111854317132384e-05 - tot_loss: 0.2141261361616671 - acc: 0.9477693144722524 - val_acc: 0.9382422802850356
Repeatition 1 Epoch 96 / 500 
 - time: 2.734746217727661 - sq_loss: 4.4274827814660966e-05 - tot_loss: 0.20995044855771994 - acc: 0.948177366702938 - val_acc: 0.9392602646759416
Repeatition 1 Epoch 97 / 500 
 - time: 2.613633394241333 - sq_loss: 4.2510422645136714e-05 - tot_loss: 0.2041906743227173 - acc: 0.9488574537540805 - val_acc: 0.9399389209365456
Repeatition 1 Epoch 98 / 500 
 - time: 2.60365891456604 - sq_loss: 4.0843046008376405e-05 - tot_loss: 0.203348161302074 - acc: 0.9494015233949945 - val_acc: 0.9406175771971497
Repeatition 1 Epoch 99 / 500 
 - time: 2.6483705043792725 - sq_loss: 3.927775105694309e-05 - tot_loss: 0.1933927623258569 - acc: 0.9494015233949945 - val_acc: 0.9412962334577536
Repeatition 1 Epoch 100 / 500 
 - time: 2.6690783500671387 - sq_loss: 3.778962854994461e-05 - tot_loss: 0.196551

Repeatition 1 Epoch 141 / 500 
 - time: 1.3088734149932861 - sq_loss: 1.320965475315461e-05 - tot_loss: 0.12386782274063535 - acc: 0.9707562568008705 - val_acc: 0.9569053274516457
Repeatition 1 Epoch 142 / 500 
 - time: 1.2869815826416016 - sq_loss: 1.3006097105972003e-05 - tot_loss: 0.11304152338851736 - acc: 0.970620239390642 - val_acc: 0.9569053274516457
Repeatition 1 Epoch 143 / 500 
 - time: 1.2888257503509521 - sq_loss: 1.2814859474019613e-05 - tot_loss: 0.11507115269046153 - acc: 0.970892274211099 - val_acc: 0.9569053274516457
Repeatition 1 Epoch 144 / 500 
 - time: 1.2799363136291504 - sq_loss: 1.2641952707781456e-05 - tot_loss: 0.12327089720523077 - acc: 0.9710282916213275 - val_acc: 0.9569053274516457
Repeatition 1 Epoch 145 / 500 
 - time: 1.2875683307647705 - sq_loss: 1.2467762644519098e-05 - tot_loss: 0.11444607965393061 - acc: 0.9715723612622416 - val_acc: 0.9569053274516457
Repeatition 1 Epoch 146 / 500 
 - time: 1.2793478965759277 - sq_loss: 1.2303407856961712e-05 - tot

Repeatition 1 Epoch 187 / 500 
 - time: 1.272984266281128 - sq_loss: 8.577537300880067e-06 - tot_loss: 0.09301329265596081 - acc: 0.9794613710554951 - val_acc: 0.9596199524940617
Repeatition 1 Epoch 188 / 500 
 - time: 1.2850959300994873 - sq_loss: 8.52281300467439e-06 - tot_loss: 0.09547662969649906 - acc: 0.9795973884657236 - val_acc: 0.9589412962334578
Repeatition 1 Epoch 189 / 500 
 - time: 1.2803444862365723 - sq_loss: 8.470853572362103e-06 - tot_loss: 0.0886829924820347 - acc: 0.9800054406964092 - val_acc: 0.9596199524940617
Repeatition 1 Epoch 190 / 500 
 - time: 1.2849688529968262 - sq_loss: 8.425413398072124e-06 - tot_loss: 0.09503645191963983 - acc: 0.9801414581066377 - val_acc: 0.9599592806243638
Repeatition 1 Epoch 191 / 500 
 - time: 1.2819106578826904 - sq_loss: 8.382793566852342e-06 - tot_loss: 0.08638911198921306 - acc: 0.9804134929270947 - val_acc: 0.9599592806243638
Repeatition 1 Epoch 192 / 500 
 - time: 1.283376693725586 - sq_loss: 8.334859558090102e-06 - tot_loss: 

...

Repeatition 1 Epoch 279 / 500 
 - time: 1.2883009910583496 - sq_loss: 6.139029210316949e-06 - tot_loss: 0.08268839694096641 - acc: 0.9857181719260065 - val_acc: 0.9630132337970818
Repeatition 1 Epoch 280 / 500 
 - time: 1.2770273685455322 - sq_loss: 6.126205789769301e-06 - tot_loss: 0.0782918478259873 - acc: 0.985582154515778 - val_acc: 0.9630132337970818
Repeatition 1 Epoch 281 / 500 
 - time: 1.293074131011963 - sq_loss: 6.110470621933928e-06 - tot_loss: 0.07302382903705507 - acc: 0.9857181719260065 - val_acc: 0.9630132337970818
Repeatition 1 Epoch 282 / 500 
 - time: 1.2870244979858398 - sq_loss: 6.091905106586637e-06 - tot_loss: 0.07617686856190886 - acc: 0.9857181719260065 - val_acc: 0.9630132337970818
Repeatition 1 Epoch 283 / 500 
 - time: 1.278618574142456 - sq_loss: 6.076143108657561e-06 - tot_loss: 0.08414178253428162 - acc: 0.9857181719260065 - val_acc: 0.9630132337970818
Repeatition 1 Epoch 284 / 500 
 - time: 1.2774534225463867 - sq_loss: 6.06061439611949e-06 - tot_loss: 0

Repeatition 1 Epoch 325 / 500 
 - time: 1.2891991138458252 - sq_loss: 5.510049504664494e-06 - tot_loss: 0.07550053940800083 - acc: 0.9873503808487486 - val_acc: 0.9647098744485918
Repeatition 1 Epoch 326 / 500 
 - time: 1.2861781120300293 - sq_loss: 5.499305189005099e-06 - tot_loss: 0.07553989753908397 - acc: 0.9872143634385201 - val_acc: 0.9647098744485918
Repeatition 1 Epoch 327 / 500 
 - time: 1.2988052368164062 - sq_loss: 5.487101134349359e-06 - tot_loss: 0.08543215162237416 - acc: 0.9872143634385201 - val_acc: 0.9643705463182898
Repeatition 1 Epoch 328 / 500 
 - time: 1.2837865352630615 - sq_loss: 5.4768265727034304e-06 - tot_loss: 0.0769850088412376 - acc: 0.9873503808487486 - val_acc: 0.9643705463182898
Repeatition 1 Epoch 329 / 500 
 - time: 1.286067008972168 - sq_loss: 5.462943136080867e-06 - tot_loss: 0.077602266876859 - acc: 0.9873503808487486 - val_acc: 0.9643705463182898
Repeatition 1 Epoch 330 / 500 
 - time: 1.2782351970672607 - sq_loss: 5.451637207443127e-06 - tot_loss:

Repeatition 1 Epoch 371 / 500 
 - time: 1.2865467071533203 - sq_loss: 5.014281668991316e-06 - tot_loss: 0.07788025033798007 - acc: 0.9881664853101197 - val_acc: 0.9647098744485918
Repeatition 1 Epoch 372 / 500 
 - time: 1.2720415592193604 - sq_loss: 5.006072569813114e-06 - tot_loss: 0.07639558342260244 - acc: 0.9880304678998912 - val_acc: 0.9650492025788938
Repeatition 1 Epoch 373 / 500 
 - time: 1.2914531230926514 - sq_loss: 4.998195890948409e-06 - tot_loss: 0.07782583007674759 - acc: 0.9880304678998912 - val_acc: 0.9657278588394977
Repeatition 1 Epoch 374 / 500 
 - time: 1.3087260723114014 - sq_loss: 4.989533863408724e-06 - tot_loss: 0.07375554346627666 - acc: 0.9884385201305768 - val_acc: 0.9660671869697998
Repeatition 1 Epoch 375 / 500 
 - time: 1.2804338932037354 - sq_loss: 4.98081089972402e-06 - tot_loss: 0.07120873097780134 - acc: 0.9884385201305768 - val_acc: 0.9653885307091958
Repeatition 1 Epoch 376 / 500 
 - time: 1.3006582260131836 - sq_loss: 4.971609087078832e-06 - tot_los

Repeatition 1 Epoch 417 / 500 
 - time: 1.2827303409576416 - sq_loss: 4.622263986675534e-06 - tot_loss: 0.06925009054339526 - acc: 0.9896626768226333 - val_acc: 0.9664065151001018
Repeatition 1 Epoch 418 / 500 
 - time: 1.295292854309082 - sq_loss: 4.614563295035623e-06 - tot_loss: 0.07828225780497533 - acc: 0.9897986942328618 - val_acc: 0.9664065151001018
Repeatition 1 Epoch 419 / 500 
 - time: 1.287093162536621 - sq_loss: 4.6059899432293605e-06 - tot_loss: 0.07154171207887394 - acc: 0.9897986942328618 - val_acc: 0.9664065151001018
Repeatition 1 Epoch 420 / 500 
 - time: 1.290116310119629 - sq_loss: 4.598641226039035e-06 - tot_loss: 0.074808887046661 - acc: 0.9897986942328618 - val_acc: 0.9667458432304038
Repeatition 1 Epoch 421 / 500 
 - time: 1.2615001201629639 - sq_loss: 4.591425295075169e-06 - tot_loss: 0.07466984073068872 - acc: 0.9897986942328618 - val_acc: 0.9664065151001018
Repeatition 1 Epoch 422 / 500 
 - time: 1.2751505374908447 - sq_loss: 4.583942882163683e-06 - tot_loss: 

Repeatition 1 Epoch 463 / 500 
 - time: 1.275484323501587 - sq_loss: 4.292160156182945e-06 - tot_loss: 0.06541956596296572 - acc: 0.9900707290533188 - val_acc: 0.9677638276213099
Repeatition 1 Epoch 464 / 500 
 - time: 1.2812838554382324 - sq_loss: 4.285197519493522e-06 - tot_loss: 0.06919444047136203 - acc: 0.9900707290533188 - val_acc: 0.9681031557516118
Repeatition 1 Epoch 465 / 500 
 - time: 1.2908968925476074 - sq_loss: 4.276742856745841e-06 - tot_loss: 0.07121937693888292 - acc: 0.9902067464635473 - val_acc: 0.9681031557516118
Repeatition 1 Epoch 466 / 500 
 - time: 1.2898802757263184 - sq_loss: 4.270318186172517e-06 - tot_loss: 0.06924153370751007 - acc: 0.9902067464635473 - val_acc: 0.9677638276213099
Repeatition 1 Epoch 467 / 500 
 - time: 1.2868432998657227 - sq_loss: 4.2632045733626e-06 - tot_loss: 0.07145273140045738 - acc: 0.9902067464635473 - val_acc: 0.9681031557516118
Repeatition 1 Epoch 468 / 500 
 - time: 1.2767341136932373 - sq_loss: 4.257227828929899e-06 - tot_loss: